In [2]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.1 MB/s eta 0:00:00


In [3]:
import numpy as np
import random
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [4]:
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'lm_head.weight', 'transformer.h.8.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
sent = '오늘 날씨가'

In [6]:
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print(input_ids)

tf.Tensor([[10070 34018]], shape=(1, 2), dtype=int32)


In [8]:
output = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        use_cache=True)
output_ids = output.numpy().tolist()[0]
print(output_ids)

[10070, 34018, 9220, 8102, 12762, 387, 11457, 8063, 8146, 13964, 7965, 9810, 10590, 30899, 17339, 9563, 8135, 9097, 8702, 9130, 10897, 21992, 18692, 20002, 9045, 7235, 9168, 11615, 8210, 10010, 7067, 9325, 20403, 9773, 11888, 8263, 9350, 26616, 9668, 10078, 15118, 8143, 9960, 13001, 9143, 9758, 48101, 9518, 9085, 9638, 7395, 9312, 6872, 6889, 26764, 28908, 10294, 9052, 12477, 8066, 12094, 9292, 9626, 14405, 9149, 9231, 6949, 8704, 11294, 7060, 17122, 16462, 8137, 9243, 29213, 7182, 23636, 27064, 26482, 13016, 9251, 18336, 10396, 8204, 9341, 25185, 13744, 8022, 25167, 9685, 11718, 37770, 10141, 9517, 15245, 25521, 9564, 32431, 8155, 9075, 12077, 9976, 9970, 9885, 14705, 21812, 10771, 12002, 8052, 8772, 9148, 10939, 9295, 19220, 21351, 20772, 10281, 9326, 35902, 9366, 16640, 16848, 15476, 14693, 18756, 9094, 8149, 9182]


In [9]:
tokenizer.decode(output_ids)

'오늘 날씨가 추워지면서, 겨울옷이 얇아지고 있습니다.\n오늘 아침 서울은 영하 4도, 대관령은 영상 2도까지 떨어졌습니다.\n낮에도 기온이 크게 오르지 못하겠는데요.\n서울의 낮 기온 5도로 어제보다 3도가량 높겠고요,\n남부지방에는 광주와 대구 10도를 보이면서 포근한 봄날씨를 되찾을 것으로 보입니다.\n다만, 동해안 지방은 건조주의보가 내려져 있어 화재 예방에 유의해야 합니다.\n지금 전국 대부분 지방에 구름이 많이 끼어있고, 제주도는 흐리고 비가 내리고 있는데요\n오후부터 밤사이 강원 영동지방에 최고 20mm의 많은 눈이 내리겠다고 기상청은 내일 예'

In [10]:
output = model(input_ids)
top10 = tf.math.top_k(output.logits[0, -1], k=10)

In [12]:
tokenizer.convert_ids_to_tokens(top10.indices.numpy())

['▁추', '▁쌀', '▁좋', '▁더', '▁많이', '▁풀', '▁예', '▁춥', '▁무척', '▁오늘']

In [13]:
sent = '오늘 날씨가'
input_ids = tokenizer.encode(sent)

while len(input_ids) < 50:
    output = model(np.array([input_ids]))
    # Top 10의 단어들을 추출
    top10 = tf.math.top_k(output.logits[0, -1], k=10)
    # Top 10의 단어들 중 랜덤으로 다음 단어로 선택.
    token_id = random.choice(top10.indices.numpy())
    input_ids.append(token_id)

tokenizer.decode(input_ids)

'오늘 날씨가 무척 추운데 내일 새벽에는 또 또 춥죠.\n지금 중부지방에서는 오늘 새벽과 비슷한 기온을 보였는데, 이 추위도 주말부터 서서히 조금씩 꺾이기겠죠 이렇게 보입니다.\n지금도 서울 등 대부분 지방은 기온이 크게 떨어지고 있는데 특히 대관령은'